In [1]:
import json
import logging
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
import faiss
import requests

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize encoder
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Ollama API endpoint
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_llm(prompt: str) -> str:
    """Query Ollama API directly"""
    payload = {
        "model": "llama3.2:1b",
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        logger.error(f"LLM query failed: {e}")
        return "Error generating response"

c:\Users\andidprastyo\Documents\Github\llm-query\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [3]:
def preprocess_record(record: Dict) -> str:
    """Convert record to searchable text"""
    formatted = []
    for k, v in record.items():
        if v:
            if isinstance(v, (int, float)):
                formatted.append(f"{k}: {v}")
            elif isinstance(v, bool):
                formatted.append(f"{k}: {'yes' if v else 'no'}")
            elif isinstance(v, str):
                v = v.strip().replace('\n', ' ')
                if v:
                    formatted.append(f"{k}: {v}")
    return " | ".join(formatted)

# Load JSONL data
with open('./fastapi/app/data/test_data.jsonl', 'r') as f:
    raw_data = [json.loads(line) for line in f if line.strip()]

# Process documents
documents = []
document_contents = []
for record in raw_data:
    content = preprocess_record(record)
    documents.append({"content": content, "metadata": record})
    document_contents.append(content)

# Create embeddings
embeddings = encoder.encode(document_contents)

# Initialize FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.astype('float32'))

print(f"Processed {len(documents)} documents")

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.27s/it]

Processed 10 documents


In [4]:
def retrieve(query: str, k: int = 3) -> List[Dict]:
    """Retrieve relevant documents"""
    try:
        query_embedding = encoder.encode([query])
        D, I = index.search(query_embedding.astype('float32'), k)
        return [documents[idx] for idx in I[0]]
    except Exception as e:
        logger.error(f"Retrieval failed: {e}")
        return []

def generate_response(query: str) -> str:
    """Generate response using RAG"""
    try:
        relevant_docs = retrieve(query)
        
        if not relevant_docs:
            return "No relevant information found."
        
        # Prepare context
        context_parts = []
        for doc in relevant_docs:
            metadata = doc['metadata']
            
            # Common fields for all queries
            common_fields = [
                'id',           # Contract ID
                'nama',         # Service Name
                'customer',     # Customer Name
                'sid',          # Service ID
                'sid_tsat',     # TSAT Service ID
                'no_wo',        # Work Order Number
                'start_kontrak',  # Contract Start
                'end_kontrak'   # Contract End
            ]
            
            # Additional fields based on query type
            if any(word in query.lower() for word in ['bank', 'banking']):
                extra_fields = ['layanan', 'segmen', 'datarate_layanan', 'project_nama']
            elif 'vsat' in query.lower():
                extra_fields = ['produk', 'layanan', 'datarate_layanan', 'uplink', 'downlink']
            else:
                extra_fields = ['layanan', 'segmen', 'provinsi', 'kabupaten']
            
            # Combine fields
            all_fields = common_fields + extra_fields
            
            # Format context with clear sections
            context_part = "SERVICE DETAILS:\n" + "\n".join(
                f"{k.replace('_', ' ').title()}: {metadata.get(k, 'N/A')}"
                for k in all_fields 
                if metadata.get(k)
            )
            context_parts.append(context_part)
        
        context = "\n\n---\n\n".join(context_parts)
        
        # Create prompt
        prompt = f"""Based on the following service and contract records:

{context}

Question: {query}

Instructions:
1. Focus on the relevant contract and service information from the context
2. Include specific details like:
   - Contract/Service IDs
   - Contract dates
   - Work order numbers
   - Customer information
3. Format the response in a clear, structured way

Answer:"""
        
        # Get response from Ollama
        response = query_llm(prompt)
        return response.strip()
        
    except Exception as e:
        logger.error(f"Response generation failed: {e}")
        return f"Failed to generate response: {str(e)}"



In [5]:
query = "Kontrak mana yang akan berakhir pada tahun 2025?"

print(f"Query: {query}")
print("Retrieving relevant documents...")
response = generate_response(query)
print(response)

Query: Kontrak mana yang akan berakhir pada tahun 2025?
Retrieving relevant documents...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.99it/s]


Berikut adalah perhitungan kontrak yang akan berakhir pada tahun 2025:

- Kita memulai dengan mengetahui id dan tanggal mulai dari kedua kontrak tersebut.
- Tanggal kontrak awal untuk kedua kontrak adalah 1 Juli 2020 dan 30 April 2022. 

Untuk menentukan kontrak berakhir pada tahun 2025, kita perlu membandingkan tanggal akhir dari kedua kontrak tersebut.

- Tanggal akhir dari kontrak ROUTER MIKROTIK CCR BANK IBK INDONESIA adalah 23 April 2025
- Tanggal akhir dari kontrak MUF Bekasi adalah 30 September 2022

Karena jumlah bulan yang berbeda antara kedua kontrak tersebut, maka kita harus membandingkan tahun kontrak untuk menentukan kontrak berakhir pada tahun 2025.

- Tahun kontrak ROUTER MIKROTIK CCR BANK IBK INDONESIA adalah 2025
- Tahun kontrak MUF Bekasi adalah 2022

Karena tahun kontrak ROUTER MIKROTIK CCR BANK IBK INDONESIA adalah tahun 2025, maka kontrak tersebut akan berakhir pada tahun 2025.
